# Phase 0: 환경 설정 및 데이터 전처리

## 📌 목적
- Google Drive 마운트 및 프로젝트 경로 설정
- 필요한 패키지 설치 (런타임당 1회)
- CSV 데이터 로드 및 표준화
- 날짜 필터링 (2024-09-01 ~ 2025-08-31) 및 중복 제거
- 인기글 원글 제거 (토픽 모델링용 데이터 생성)
- 전처리된 데이터를 Parquet 형식으로 저장

## 📂 출력 파일
```
outputs/
├── 익게2_for_topic.parquet
├── 자유게시판_for_topic.parquet
├── 연애상담소_for_topic.parquet
├── 익게1_for_topic.parquet
└── 인기글_clean.parquet
```

## ⏱️ 예상 소요 시간
- 패키지 설치: 5-10분 (처음 실행 시)
- 데이터 전처리: 1-2분

## 🔗 다음 단계
이 노트북 완료 후 → `phase1_topic_modeling.ipynb` 실행


---
## 1. Google Drive 마운트


In [1]:
from google.colab import drive

# Google Drive 마운트 (이미 마운트되어 있다면 건너뛰어도 됩니다)
drive.mount('/content/drive', force_remount=True)

print("✅ Google Drive 마운트 완료")


Mounted at /content/drive
✅ Google Drive 마운트 완료


---
## 2. 프로젝트 경로 설정

> ⚠️ `PROJECT_ROOT`가 실제 데이터와 모델이 저장된 Google Drive 경로와 다르다면 반드시 수정해주세요.


In [2]:
from pathlib import Path

# 프로젝트 루트 (필요 시 사용자 환경에 맞게 변경)
PROJECT_ROOT = Path("/content/drive/MyDrive/board_crawling")

# 주요 디렉토리 경로
DATA_DIR = PROJECT_ROOT / "data"          # 원본 CSV 위치
OUTPUT_DIR = PROJECT_ROOT / "outputs"      # 전처리 결과 저장 위치
MODEL_DIR = PROJECT_ROOT / "models"        # 학습 모델 및 체크포인트 저장 위치

# 디렉토리 생성
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)

print(f"📁 PROJECT_ROOT : {PROJECT_ROOT}")
print(f"📁 DATA_DIR     : {DATA_DIR}")
print(f"📁 OUTPUT_DIR   : {OUTPUT_DIR}")
print(f"📁 MODEL_DIR    : {MODEL_DIR}")


📁 PROJECT_ROOT : /content/drive/MyDrive/board_crawling
📁 DATA_DIR     : /content/drive/MyDrive/board_crawling/data
📁 OUTPUT_DIR   : /content/drive/MyDrive/board_crawling/outputs
📁 MODEL_DIR    : /content/drive/MyDrive/board_crawling/models


---
## 3. 필수 패키지 설치

- 런타임이 초기화될 때마다 한 번씩만 실행하면 됩니다.
- 이미 설치되어 있다면 건너뛰어도 무방합니다.


In [3]:
# Phase 0에서는 전처리용 기본 패키지만 설치합니다.
# - pandas, numpy, pyarrow: CSV/Parquet 입출력 및 데이터 처리용

%pip install -q pandas numpy pyarrow

print("✅ Phase 0 기본 파이썬 패키지 설치 완료")


✅ Phase 0 기본 파이썬 패키지 설치 완료


In [4]:
# Phase 0에서는 Mecab 시스템 사전을 설치하지 않습니다.
# - 형태소 분석이 필요한 Phase 1 / Phase 2에서 각각 설치하도록 분리했습니다.
pass


---
## 4. 라이브러리 import 및 Mecab 초기화


In [5]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

print("✅ 기본 라이브러리 import 완료")


✅ 기본 라이브러리 import 완료


In [6]:
# Phase 0에서는 Mecab을 직접 사용하지 않습니다.
# - 이후 Phase 1 / Phase 2에서 각자 필요한 방식으로 Mecab을 초기화합니다.


---
## 5. 데이터 처리 유틸 함수


In [7]:
def extract_id_from_url(u: str):
    """URL에서 6자리 이상 숫자를 추출하여 post id로 활용"""
    if pd.isna(u):
        return np.nan
    m = re.search(r"(\d{6,})", str(u))
    return int(m.group(1)) if m else np.nan

STD_COLS_BASE = [
    "id", "title", "url", "date",
    "likes", "views",
    "contents", "comments", "comments_num",
]

def load_csv(path: Path, has_board: bool = False) -> pd.DataFrame:
    """CSV를 표준 스키마로 로드 + 기본 정제"""
    print(f"\n📝 Loading: {path.name}")
    df = pd.read_csv(path)

    cols = STD_COLS_BASE.copy()
    if has_board:
        cols.insert(4, "board")

    for c in cols:
        if c not in df.columns:
            df[c] = np.nan

    df = df[cols].copy()

    df["title"] = df["title"].astype(str).fillna("")
    df["url"] = df["url"].astype(str).fillna("")
    df["contents"] = df["contents"].astype(str).fillna("")
    df["comments"] = df["comments"].astype(str).fillna("")

    if has_board:
        df["board"] = df["board"].astype(str).fillna("")

    for c in ["likes", "views", "comments_num"]:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)

    df["id"] = pd.to_numeric(df["id"], errors="coerce")
    backup_id = df["url"].apply(extract_id_from_url)
    df["post_key"] = df["id"].fillna(backup_id).astype("Int64")

    dt = pd.to_datetime(df["date"], errors="coerce", utc=True)
    dt = dt.dt.tz_convert("Asia/Seoul").dt.tz_localize(None)
    df["date"] = dt

    print(f"  ✓ {len(df):,}행 로드 | date dtype: {df['date'].dtype}")
    return df

START_DT = pd.Timestamp(2024, 9, 1)
END_DT = pd.Timestamp(2025, 8, 31, 23, 59, 59)

def filter_date_and_dedup(df: pd.DataFrame, name: str) -> pd.DataFrame:
    before = len(df)
    mask = (df["date"] >= START_DT) & (df["date"] <= END_DT)
    df = df.loc[mask].copy()
    after_date = len(df)

    if "post_key" in df.columns:
        df = df.sort_values("date").drop_duplicates(subset=["post_key"], keep="first")
    else:
        df = df.sort_values("date").drop_duplicates(subset=["url"], keep="first")

    after_dedup = len(df)
    print(f"[{name}] {before:,} → 날짜 {after_date:,} → 중복 {after_dedup:,}")
    return df

def ensure_post_key(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if "post_key" not in df.columns:
        df["id"] = pd.to_numeric(df["id"], errors="coerce")
        backup = df["url"].apply(extract_id_from_url)
        df["post_key"] = df["id"].fillna(backup).astype("Int64")
    return df

print("✅ 유틸 함수 정의 완료")


✅ 유틸 함수 정의 완료


---
## 6. CSV 데이터 로드


In [8]:
# 일반 게시판 로드
ik2_df = load_csv(DATA_DIR / "익게2_full_details.csv")
free_df = load_csv(DATA_DIR / "자유게시판_full_details.csv")
love_df = load_csv(DATA_DIR / "연애상담소_full_details.csv")
ik1_df = load_csv(DATA_DIR / "익게1_full_details.csv")

# 인기글 로드 (board 포함)
hot_df = load_csv(DATA_DIR / "인기글_full_details.csv", has_board=True)

print("\n✅ 모든 CSV 로드 완료")



📝 Loading: 익게2_full_details.csv
  ✓ 64,608행 로드 | date dtype: datetime64[ns]

📝 Loading: 자유게시판_full_details.csv
  ✓ 2,500행 로드 | date dtype: datetime64[ns]

📝 Loading: 연애상담소_full_details.csv
  ✓ 2,000행 로드 | date dtype: datetime64[ns]

📝 Loading: 익게1_full_details.csv
  ✓ 1,860행 로드 | date dtype: datetime64[ns]

📝 Loading: 인기글_full_details.csv
  ✓ 7,339행 로드 | date dtype: datetime64[ns]

✅ 모든 CSV 로드 완료


---
## 7. 날짜 필터링 및 중복 제거


In [9]:
print("\n📊 날짜 필터링 및 중복 제거 진행 중...\n")

ik2_df = filter_date_and_dedup(ik2_df, "익게2")
free_df = filter_date_and_dedup(free_df, "자유게시판")
love_df = filter_date_and_dedup(love_df, "연애상담소")
ik1_df = filter_date_and_dedup(ik1_df, "익게1")
hot_df = filter_date_and_dedup(hot_df, "인기글(raw)")

print("\n✅ 필터링 완료")



📊 날짜 필터링 및 중복 제거 진행 중...

[익게2] 64,608 → 날짜 57,261 → 중복 54,662
[자유게시판] 2,500 → 날짜 2,466 → 중복 2,466
[연애상담소] 2,000 → 날짜 1,978 → 중복 1,978
[익게1] 1,860 → 날짜 1,853 → 중복 1,853
[인기글(raw)] 7,339 → 날짜 7,260 → 중복 7,260

✅ 필터링 완료


---
## 8. 인기글 게시판 정보 정제


In [10]:
def clean_board_name(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = str(s).strip().strip("|")
    s = re.sub(r"[\[\]\(\)]", "", s)
    return s.strip()

hot_df["board"] = hot_df["board"].astype(str).apply(clean_board_name)

ALLOWED_BOARDS = ["자유게시판", "연애상담소", "익게1", "익게2"]
hot_df = hot_df[hot_df["board"].isin(ALLOWED_BOARDS)].copy()

print("\n📊 인기글 게시판 분포:")
print(hot_df["board"].value_counts())
print(f"\n✅ 정제된 인기글 수: {len(hot_df):,}")



📊 인기글 게시판 분포:
board
익게2      6168
자유게시판     181
연애상담소     126
익게1       117
Name: count, dtype: int64

✅ 정제된 인기글 수: 6,592


---
## 9. post_key 재확인


In [11]:
ik2_df = ensure_post_key(ik2_df)
free_df = ensure_post_key(free_df)
love_df = ensure_post_key(love_df)
ik1_df = ensure_post_key(ik1_df)
hot_df = ensure_post_key(hot_df)

print("✅ 모든 데이터프레임에서 post_key 보강 완료")


✅ 모든 데이터프레임에서 post_key 보강 완료


---
## 10. 인기글 원글 제거


In [12]:
hot_keys = set(hot_df["post_key"].dropna().astype("Int64"))
print(f"📌 인기글 고유 post_key 수: {len(hot_keys):,}")


def mask_hot(df: pd.DataFrame, name: str) -> pd.DataFrame:
    before = len(df)
    out = df[~df["post_key"].isin(hot_keys)].copy()
    removed = before - len(out)
    print(f"[{name}] {before:,} → {len(out):,} (인기글 제거 {removed:,}개)")
    return out

ik2_for_topic = mask_hot(ik2_df, "익게2")
free_for_topic = mask_hot(free_df, "자유게시판")
love_for_topic = mask_hot(love_df, "연애상담소")
ik1_for_topic = mask_hot(ik1_df, "익게1")

print("\n✅ 인기글 원글 제거 완료")


📌 인기글 고유 post_key 수: 6,592
[익게2] 54,662 → 49,251 (인기글 제거 5,411개)
[자유게시판] 2,466 → 2,285 (인기글 제거 181개)
[연애상담소] 1,978 → 1,852 (인기글 제거 126개)
[익게1] 1,853 → 1,736 (인기글 제거 117개)

✅ 인기글 원글 제거 완료


---
## 11. 전처리 결과 저장


In [13]:
print("\n💾 전처리 데이터 저장 중...")

ik2_for_topic.to_parquet(OUTPUT_DIR / "익게2_for_topic.parquet", index=False)
free_for_topic.to_parquet(OUTPUT_DIR / "자유게시판_for_topic.parquet", index=False)
love_for_topic.to_parquet(OUTPUT_DIR / "연애상담소_for_topic.parquet", index=False)
ik1_for_topic.to_parquet(OUTPUT_DIR / "익게1_for_topic.parquet", index=False)
hot_df.to_parquet(OUTPUT_DIR / "인기글_clean.parquet", index=False)

print("  ✓ 익게2_for_topic.parquet 저장")
print("  ✓ 자유게시판_for_topic.parquet 저장")
print("  ✓ 연애상담소_for_topic.parquet 저장")
print("  ✓ 익게1_for_topic.parquet 저장")
print("  ✓ 인기글_clean.parquet 저장")
print(f"\n✅ 모든 파일 저장 완료 → {OUTPUT_DIR}")



💾 전처리 데이터 저장 중...
  ✓ 익게2_for_topic.parquet 저장
  ✓ 자유게시판_for_topic.parquet 저장
  ✓ 연애상담소_for_topic.parquet 저장
  ✓ 익게1_for_topic.parquet 저장
  ✓ 인기글_clean.parquet 저장

✅ 모든 파일 저장 완료 → /content/drive/MyDrive/board_crawling/outputs


---
## 12. 전처리 요약 및 다음 단계


In [14]:
summary_rows = [
    {"게시판": "익게2", "전처리 후": f"{len(ik2_for_topic):,}",
     "날짜 범위": f"{ik2_for_topic['date'].min():%Y-%m-%d} ~ {ik2_for_topic['date'].max():%Y-%m-%d}"},
    {"게시판": "자유게시판", "전처리 후": f"{len(free_for_topic):,}",
     "날짜 범위": f"{free_for_topic['date'].min():%Y-%m-%d} ~ {free_for_topic['date'].max():%Y-%m-%d}"},
    {"게시판": "연애상담소", "전처리 후": f"{len(love_for_topic):,}",
     "날짜 범위": f"{love_for_topic['date'].min():%Y-%m-%d} ~ {love_for_topic['date'].max():%Y-%m-%d}"},
    {"게시판": "익게1", "전처리 후": f"{len(ik1_for_topic):,}",
     "날짜 범위": f"{ik1_for_topic['date'].min():%Y-%m-%d} ~ {ik1_for_topic['date'].max():%Y-%m-%d}"},
    {"게시판": "인기글", "전처리 후": f"{len(hot_df):,}",
     "날짜 범위": f"{hot_df['date'].min():%Y-%m-%d} ~ {hot_df['date'].max():%Y-%m-%d}"},
]

summary_df = pd.DataFrame(summary_rows)
print("\n" + "="*70)
print("Phase 0 완료 - 데이터 전처리 요약".center(70))
print("="*70)
print(summary_df.to_string(index=False))
print("="*70)
print("\n🎉 Phase 0 완료!")
print("다음 단계: phase1_topic_modeling.ipynb 실행")



                       Phase 0 완료 - 데이터 전처리 요약                        
  게시판  전처리 후                   날짜 범위
  익게2 49,251 2024-09-01 ~ 2025-08-31
자유게시판  2,285 2024-09-01 ~ 2025-08-31
연애상담소  1,852 2024-09-01 ~ 2025-08-31
  익게1  1,736 2024-09-01 ~ 2025-08-31
  인기글  6,592 2024-09-01 ~ 2025-08-31

🎉 Phase 0 완료!
다음 단계: phase1_topic_modeling.ipynb 실행
